In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px


sns.set(style="whitegrid")

df = gpd.read_file("C://Users//shoaib//Downloads//merged_phu_analysis.geojson")  # Adjust filename
# === Normalize inputs (0 to 1 scale) ===
scaler = MinMaxScaler()
df['ed_rate_norm'] = scaler.fit_transform(df[['avg_ed_rate']])
df['deprivation_norm'] = scaler.fit_transform(df[['material_resources_HUID']])
df['facilities_norm'] = scaler.fit_transform(df[['facilities_per_10k']])
df['low_facilities_norm'] = 1 - df['facilities_norm']

# Compute Composite Risk Score 
df['risk_score'] = df['ed_rate_norm'] + df['deprivation_norm'] + df['low_facilities_norm']
df['risk_score_norm'] = scaler.fit_transform(df[['risk_score']])
print(df)

# Sort and find top 5
df = df.sort_values(by='risk_score_norm', ascending=True)
top5 = df.nlargest(5, 'risk_score_norm')['HU_NAME'].tolist()

# Color for top 5
df['color'] = df['HU_NAME'].apply(lambda x: 'Top 5' if x in top5 else 'Other')

# Create interactive horizontal bar chart 
fig = px.bar(
    df,
    x='risk_score_norm',
    y='HU_NAME',
    orientation='h',
    color='color',
    color_discrete_map={'Top 5': 'crimson', 'Other': 'gray'},
    hover_data={
        'HU_NAME': True,
        'risk_score_norm': ':.2f',
        'ed_rate_norm': ':.2f',
        'deprivation_norm': ':.2f',
        'low_facilities_norm': ':.2f'
    },
    labels={
        'HU_NAME': 'Public Health Unit',
        'risk_score_norm': 'Composite Risk Score (Normalized)',
        'ed_rate_norm': 'ED Visit Risk',
        'deprivation_norm': 'Deprivation Risk',
        'low_facilities_norm': 'Facility Shortage Risk',
    },
    title='📊 Composite Risk Score by PHU — Interactive Highlighting of Top 5'
)

fig.update_layout(
    yaxis=dict(autorange='reversed'),  # Highest risk on top
    height=900,
    xaxis_title='Normalized Composite Risk Score (0–1)',
    yaxis_title='',
    showlegend=False
)

# Show plot locally 
fig.show()


    FID    OGF_ID  PHU_ID                                           NAME_ENG  \
0     1  71878062    2270                          York Region Public Health   
1     2  71878005    5183                            Huron Perth Health Unit   
2     3  71878059    2265                  Region of Waterloo, Public Health   
3     4  71878029    4913                         Southwestern Public Health   
4     5  71878057    2237                    Hamilton Public Health Services   
5     6  71878051    2262                   Thunder Bay District Health Unit   
6     7  71878041    2253                                 Peel Public Health   
7     8  71878038    2242                              Lambton Public Health   
8     9  71878035    2266             Wellington-Dufferin-Guelph Health Unit   
9    10  71878032    2227                           Brant County Health Unit   
10   11  71878026    2244                       Middlesex-London Health Unit   
11   12  71878023    2261               